In [2]:
# N 일 기준 k 배 이상
import datetime
import pymysql
import pandas as pd

def getTable(day,n):
    
    conn= pymysql.connect(
           *
            )

    cursor = conn.cursor()
    
    sql =   f"""select avg_table.stockname, vol/avg_vol as threshold, vol from
        (select stockname, avg(vol) as avg_vol
        from (select * from (select * , rank() over(PARTITION BY stockname order by stock_date desc) as ranking from dailyinfo where stock_date < {day}) as A where ranking <= {n})
        as B 
        group by stockname) as avg_table
        inner join (select stockname, vol from dailyinfo where stock_date = {day}) as day_table
        on avg_table.stockname = day_table.stockname
        order by threshold desc;"""
    
    cursor.execute(sql)
    
    result = list(cursor.fetchall())

    conn.close()
    
    return result


# _days = str(datetime.date.today())
_days = "2022-05-20"
    
df = pd.DataFrame(getTable("'"+_days+"'",20),columns=['stockname','times','vol'])

df = df[df['times'] > 5]

df['stock_day'] = _days

from sqlalchemy import create_engine

pymysql.install_as_MySQLdb()
import MySQLdb
            
engine = create_engine("*",encoding='utf-8')
conn = engine.connect()

df.to_sql(name='volumeup', con = engine, if_exists = 'replace' ,index = False)
            
conn.close()